In [1]:
import numpy as np
import pickle
import skimage.transform
import scipy

import theano
import theano.tensor as T


In [5]:
from nn import ConvLayer
from theano.tensor.signal import pool

class vgg19(object):
    def __init__(self, input):
        batch_size = 1
        width_1 = input.shape[0]
        rng=np.random.RandomState(11)


        self.conv1_1 = ConvLayer(rng,
            input=input,
            image_shape=(batch_size, 3, width_1, width_1),
            filter_shape=(64, 3, 3, 3),
            border_mode=1
        )
        self.conv1_2 = ConvLayer(
            rng,
            input=self.conv1_1.output,
            image_shape=(batch_size, 64, width_1, width_1),
            filter_shape=(64, 64, 3, 3),
            border_mode=1
        )
        self.pool1 = pool.pool_2d(
            input=self.conv1_2.output,
            ds=(2,2)
        )
        width_2 = width_1/2


        self.conv2_1 = ConvLayer(
            rng,
            input=self.pool1,
            image_shape=(batch_size, 64, width_2, width_2),
            filter_shape=(128, 64, 3, 3),
            border_mode=1
        )
        self.conv2_2  = ConvLayer(
            rng,
            input=self.conv2_1.output,
            image_shape=(batch_size, 128, width_2, width_2),
            filter_shape=(128, 128, 3, 3),
            border_mode=1
        )
        self.pool2 = pool.pool_2d(
                input=self.conv2_2.output,
                ds=(2,2)
        )    
        width_3 = width_2/2
        self.conv3_1 = ConvLayer(
            rng,
            input=self.pool2,
            image_shape=(batch_size, 128, width_3, width_3),
            filter_shape=(256, 128, 3, 3),
            border_mode=1
        )
        self.conv3_2 = ConvLayer(
            rng,
            input=self.conv3_1.output,
            image_shape=(batch_size, 256, width_3, width_3),
            filter_shape=(256, 256, 3, 3),
            border_mode=1
        )
        self.conv3_3 = ConvLayer(
            rng,
            input=self.conv3_2.output,
            image_shape=(batch_size, 256, width_3, width_3),
            filter_shape=(256, 256, 3, 3),
            border_mode=1
        )

        self.conv3_4 = ConvLayer(
            rng,
            input=self.conv3_3.output,
            image_shape=(batch_size, 256, width_3, width_3),
            filter_shape=(512, 256, 3, 3),
            border_mode=1
        )
        self.pool3 = pool.pool_2d(
                input=self.conv3_4.output,
                ds=(2,2)
        ) 
        width_4 = width_3/2
        self.conv4_1 = ConvLayer(
            rng,
            input=self.pool3,
            image_shape=(batch_size, 512, width_4, width_4),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )
        self.conv4_2 = ConvLayer(
            rng,
            input=self.conv4_1.output,
            image_shape=(batch_size, 512, width_4, width_4),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )
        self.conv4_3 = ConvLayer(
            rng,
            input=self.conv4_2.output,
            image_shape=(batch_size, 512, width_4, width_4),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )

        self.conv4_4 = ConvLayer(
            rng,
            input=self.conv4_3.output,
            image_shape=(batch_size, 512, width_4, width_4),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )
        self.pool4 = pool.pool_2d(
                input=self.conv4_4.output,
                ds=(2,2)
        ) 
        width_5 = width/2
        self.conv5_1 = ConvLayer(
            rng,
            input=self.pool4,
            image_shape=(batch_size, 512, width_5, width_5),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )
        self.conv5_2 = ConvLayer(
            rng,
            input=self.conv5_1.output,
            image_shape=(batch_size, 512, width_5, width_5),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )
        self.conv5_3 = ConvLayer(
            rng,
            input=self.conv5_2.output,
            image_shape=(batch_size, 512, width_5, width_5),
            filter_shape=(512, 512, 3, 3),
            border_mode=1
        )

        self.conv5_4 = ConvLayer(
            rng,
            input=self.conv5_3.output,
            image_shape=(batch_size, 512, width_5, width_5),
            filter_shape=(512, 256, 3, 3),
            border_mode=1
        )
        self.pool5 = pool.pool_2d(
                input=self.conv5_4.output,
                ds=(2,2)
        ) 


In [12]:

# build VGG net and load weights

# net = build_model()

values = pickle.load(open('vgg19_normalized.pkl'))['param values']
np.matrix(values).shape
# lasagne.layers.set_all_param_values(net['pool5'], values)

(1, 32)

In [ ]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (IMAGE_W, w*IMAGE_W/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*IMAGE_W/w, IMAGE_W), preserve_range=True)

    # Central crop
    h, w, _ = im.shape
    im = im[h//2-IMAGE_W//2:h//2+IMAGE_W//2, w//2-IMAGE_W//2:w//2+IMAGE_W//2]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert RGB to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [6]:

photo = plt.imread('data/test/Tuebingen_Neckarfront.jpg')


NameError: name 'plt' is not defined

In [2]:
# !wget https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg19_normalized.pkl


--2016-12-08 14:30:40--  https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/vgg19_normalized.pkl
Resolving s3.amazonaws.com... 52.216.64.219
Connecting to s3.amazonaws.com|52.216.64.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80126892 (76M) [binary/octet-stream]
Saving to: ‘vgg19_normalized.pkl’

vgg19_normalized.pk 100%[===================>]  76.41M  15.2MB/s    in 5.2s    

2016-12-08 14:30:45 (14.7 MB/s) - ‘vgg19_normalized.pkl’ saved [80126892/80126892]



In [ ]:
def build_model():
    net = {}
    net['input'] = InputLayer((1, 3, IMAGE_W, IMAGE_W))
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1, flip_filters=False)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['conv1_2'], 2, mode='average_exc_pad')
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1, flip_filters=False)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['conv2_2'], 2, mode='average_exc_pad')
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1, flip_filters=False)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1, flip_filters=False)
    net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3, pad=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['conv3_4'], 2, mode='average_exc_pad')
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1, flip_filters=False)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1, flip_filters=False)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1, flip_filters=False)
    net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3, pad=1, flip_filters=False)
    net['pool4'] = PoolLayer(net['conv4_4'], 2, mode='average_exc_pad')
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1, flip_filters=False)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1, flip_filters=False)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1, flip_filters=False)
    net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3, pad=1, flip_filters=False)
    net['pool5'] = PoolLayer(net['conv5_4'], 2, mode='average_exc_pad')

    return net


In [4]:
values = pickle.load(open('vgg19_normalized.pkl'))['param values']